In [2]:
from hwr.data.generator import IAMSequence
from hwr.constants import SPLIT
from hwr.models.ONNET import ONNET
from hwr.decoding.ctc_decoder import BestPathDecoder, TrieBeamSearchDecoder
from matplotlib import pyplot as plt
import numpy as np

In [3]:
model = ONNET(preload=True, gru=False, gpu=True)
model.get_model_summary()

preloading model weights from /home/jasper/Desktop/fyp/HWR/hwr/../models/iamon/ONNET/pretrained-deep-lstm/weights.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
xs (InputLayer)                 (None, None, 6)      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 60)     2580        xs[0][0]                         
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 60)     240         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, 60)     0           batch_normalization[0][0] 

In [4]:
p = 6
train_seq = IAMSequence(SPLIT.TRAIN, batch_size=20, preprocess=p,
                                   npz=True, inout_ratio=4)
val1_seq = IAMSequence(SPLIT.VAL1, batch_size=100, preprocess=p,
                                  npz=True, inout_ratio=4)
eval_seq = IAMSequence(SPLIT.TEST, batch_size=100, preprocess=p,
                      npz=True, pred=True, pad_to=(1200, 300))

In [5]:
model.train(train_seq, val1_seq, epochs=3000, earlystop=10)

Epoch 1/3000
 19/269 [=>............................] - ETA: 1:12 - loss: 5.0801

KeyboardInterrupt: 

In [7]:
#TrieBeamSearchDecoder(25, lm="sbo", ngram=7, prune=10, trie="100k", gamma=1)
model.evaluate(eval_seq, decoder=BestPathDecoder())

39/39 [==============================] - 6s 148ms/step


{'character_error_rate': 0.08255496354932794,
 'word_error_rate': 0.3170504844629366}

In [6]:
model.load_weights("../models/iamon/checkpoint/ONNET/2019-09-29-13:04:27/weights.h5", full_path=True)